# Geometrical Methods in Machine Learning
## Seminar 1: PCA

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
from math import isclose

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine, load_boston, fetch_olivetti_faces, fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

## 1. Model data

Consider an artificial dataset which is sampled from multivariate normal distribution with covariance matrix $C = \begin{pmatrix} 3 & 1 \\ 1 & 2 \end{pmatrix}$

In [ ]:
sample_size = 500

mu = np.zeros(2)
C = np.array([[3,1],[1,2]])
data = np.random.multivariate_normal(mu, C, size=sample_size)

Find true PC components by covariance matrix diagonalization:

$$S_{TRUE} = \mathbf{V}\mathbf{C}\mathbf{V}^-1$$

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(C)

print("True eigenvalues:\n", eigenvalues)
print("\nTrue eigenvectors:\n", eigenvectors)

Find principal components using PCA from `scikit-learn`:

In [ ]:
# estimate principal components from empirical data
pca = PCA()
pca.fit(data)

eigenvectors_hat = pca.components_
eigenvalues_hat = pca.explained_variance_

print("Eigenvalues:\n", eigenvalues_hat)
print("\nEigenvectors:\n", eigenvectors_hat)

In [ ]:
def plot_principal_components(data, model, scatter=True):
    W_pca = model.components_
    
    if scatter:
        plt.scatter(data[:,0], data[:,1])
    
    plt.plot(data[:,0], -(W_pca[0,0]/W_pca[0,1])*data[:,0], color="b", alpha=0.5, label="Estimated PCs")
    plt.plot(data[:,0], -(W_pca[1,0]/W_pca[1,1])*data[:,0], color="b", alpha=0.5)

    plt.axis('equal')
    limits = [np.minimum(np.amin(data[:,0]), np.amin(data[:,1]))-0.5,
              np.maximum(np.amax(data[:,0]), np.amax(data[:,1]))+0.5]
    plt.xlim(limits[0],limits[1])
    plt.ylim(limits[0],limits[1])
    plt.draw()

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.xlim(-7, 7)
plt.ylim(-7, 7)
plt.grid(linestyle="dotted")

# plot true principal components
plt.plot(data[:,0], (eigenvectors[0,0] / eigenvectors[0,1]) * data[:,0], color="r", alpha=0.5, label="True PCs")
plt.plot(data[:,0], (eigenvectors[1,0] / eigenvectors[1,1]) * data[:,0], color="r", alpha=0.5)

# plot estimated principal components
plot_principal_components(data, pca, scatter=False)

plt.title("Multivariate gaussian")
plt.xlabel("X")
plt.ylabel("Y")
plt.scatter(data[:,0], data[:,1], alpha=0.25)
plt.legend()
plt.show()

#### Exercise

Find principal components manually as SVD solution of PCA problem.

Estimator of covariance matrix of normally distributed variables:

$$\hat{S}_{\mathcal{N}} = \frac{1}{n} (\mathbf{X} - \mathbf{\bar{x}})^T (\mathbf{X} - \mathbf{\bar{x}})$$

In [ ]:
# center the data
mean = # your code here
data_centered = # your code here

# estimate data covariance matrix, X.T * X
covariance_hat = # your code here

Use SVD `np.linalg.eig` of centered data matrix $\mathbf{X}^T$:

$$\mathbf{X} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T$$

In [ ]:
# find eigenvalues and eigenvectors of data covariance matrix via SVD

# your code here

print("Numerical mean of centered data (should be close zero):\n", mean)
print("\nEstimation of covariance matrix:\n", covariance_hat)
print("\nEigenvectors:\n", eigenvectors)

Or eigendecomposition `np.linalg.eig` of covariance matrix $\mathbf{X}^T\mathbf{X}$:

$$\mathbf{X}^T\mathbf{X} = \mathbf{V} \mathbf{\Lambda} \mathbf{V}^{-1}$$

In [ ]:
# find eigenvalues and eigenvectors of data covariance matrix via eigendecomposition

# your code here

print("Numerical mean of centered data (should be close zero):\n", mean)
print("\nEstimation of covariance matrix:\n", covariance_hat)
print("\nEigenvectors:\n", eigenvectors)

#### Exercise:

Compute and plot the projection onto the subspace spanned by eigenvectors.

Let $\mathbf{X}$ be $n \times D$ centered data matrix whose columns contain the different observations. Let $\mathbf{V}$ be $D \times d$ matrix whose columns contain $d$ largest eigenvectors. Then the projected data $\mathbf{Z}$ is obtained as projection to the subspace spanned by $d$ largest eigenvectors $\mathbf{Z} = \mathbf{X} \mathbf{V}$.

New basis of data is called eigenbasis. Principal components are axes in the new eigenbasis.

In [ ]:
# compute projection to eigenbasis
proj = # your code here

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.xlim(-7, 7)
plt.ylim(-7, 7)
plt.grid(linestyle="dotted")

plt.title("Projected data")
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.scatter(proj[:,0], proj[:,1], alpha=0.25)
plt.show()

#### Exercise:

Reduce the dimensionality of the data by the projection onto the subspace spanned by first eigenvector. Plot it.

In [ ]:
# reduce the dimensionality of the data
proj = # your code here

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.xlim(-7, 7)
plt.ylim(-7, 7)
plt.grid(linestyle="dotted")

plt.title("Projected data")
plt.xlabel("PC 1")
plt.scatter(proj, np.zeros(proj.shape[0]), alpha=0.25)
plt.show()

Let $\mathbf{Z}$ be $n \times d$ centered data matrix with $n$ observations with $d$ features with sample mean $\mathbf{\bar{x}}$. Let $\mathbf{V}$ be $D \times d$ projection matrix whose columns contain $d$ largest eigenvectors.

Then PCA reconstruction is defined:

$$\mathbf{\tilde{X}} = \mathbf{Z}\mathbf{V}^T + \mathbf{\bar{x}}$$

#### Exercise:

Reconstuct reduced dimensionality data to original space.

In [ ]:
# reconstuct the data
data_reconstructed = # your code here

In [ ]:
fig = plt.figure(figsize=(6,6))
plt.xlim(-7, 7)
plt.ylim(-7, 7)
plt.grid(linestyle="dotted")

plt.title("Reconstructed data")
plt.xlabel("X")
plt.ylabel("Y")
plt.scatter(data_reconstructed[:,0], data_reconstructed[:,1], alpha=0.25)
plt.show()

## 2. Real data

### 2.1 Airfoils

In [ ]:
ref_points = np.loadtxt('./data/ref_points.csv', delimiter=',') # X coordinates
X_train = np.loadtxt('./data/airfoils.csv', delimiter=',') # 199 wings
X_test = np.loadtxt('./data/test_afl.csv', delimiter=',') # 200th wing

In [ ]:
ref_points

In [ ]:
fig = plt.figure(figsize=(12,5))
plt.ylim(-0.23, 0.23)
plt.grid(linestyle="dotted")

#plt.plot(ref_points, test_point, '-*', label = 'Original test')
plt.plot(ref_points, X_train[0, :], '-*', label = 'Airfoil #1')
plt.plot(ref_points, X_train[1, :], '-*', label = 'Airfoil #2')
plt.plot(ref_points, X_train[2, :], '-*', label = 'Airfoil #3')
#plt.plot(ref_points, test_point, label = 'Original')

plt.title("Airfoils")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(['Airfoil #1', 'Airfoil #2', 'Airfoil #3'])

### Airfoils principal components

#### Exercise  
Plot eigenvectors for the 5 largest and 5 smallest eigenvalues

In [ ]:
# apply PCA

# your code here
components = 

In [ ]:
for i in range(5):
    fig = plt.figure(figsize=(6,2.5))
    plt.ylim(-0.3, 0.3)
    plt.grid(linestyle="dotted")
    plt.plot(ref_points, pca.components_[i, :], '-*')
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend(['Component #{}'.format(i+1)])
    plt.show()
    
for i in range(len(pca.components_) - 5, len(pca.components_)):
    fig = plt.figure(figsize=(6,2.5))
    plt.ylim(-0.3, 0.3)
    plt.grid(linestyle="dotted")
    plt.plot(ref_points, pca.components_[i, :], '-*')
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend(['Component #{}'.format(i+1)])
    plt.show()

#### Exercise

Explained variance of $i$-th principal component is the ratio of corresponding eigenvalue to the sum of all eigenvalues:

$$\mathrm{EV_i} := \frac{\lambda_i}{\lambda_1 + \dots + \lambda_D} = \frac{\lambda_i}{\sum_{j=1}^{D} \lambda_j}$$

Cumulative explained variance of the first $d$ principal components is the ratio of the sum of corresponding eigenvalues to the sum of all eigenvalues:

$$\mathrm{CEV_d} := \frac{\sum_{i=1}^{d} \lambda_i}{\sum_{j=1}^{D} \lambda_j}$$

Plot expained variance and cumulative explained variance for all $D$ principal components, choose sample dimensionality, state the rule used.

In [ ]:
def EV_i(i, eigenvalues):
    # your code here

In [ ]:
def CEV_d(d, eigenvalues):
    # your code here

In [ ]:
EV = []
CEV = []

for i in range(X_train.shape[1]):
    EV.append(EV_i(i, pca.explained_variance_))
    CEV.append(CEV_d(i, pca.explained_variance_))

In [ ]:
# plot EV/CEVs
fig = plt.figure(figsize=(12,5.25))

plt.subplot(121)
plt.title("Explained variance")
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.plot(EV, "o-")

plt.subplot(122)
plt.title("Cumulative explained variance")
plt.axhline(linewidth=1, y=0.99, color='r')
plt.axhline(linewidth=1, y=0.95, color='r')
plt.axhline(linewidth=1, y=0.9, color='r')
plt.axhline(linewidth=1, y=0.8, color='r')
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.plot(CEV, "o-")

#### Exercise 
Describe original airfoil vector in eigenbasis and reproject back for visualization. Try different number of basis eigenvectors.

In [ ]:
# apply PCA

# your code here

X_test_pcs = # your code here, should be vector of original dimension

In [ ]:
fig = plt.figure(figsize=(12,5))
plt.ylim(-0.23, 0.23)
plt.grid(linestyle="dotted")

#plt.plot(ref_points, test_point, '-*', label = 'Original test')
plt.plot(ref_points, X_test, '-*', label = 'Airfoil #200')
plt.plot(ref_points, X_test_pcs, '-*', label = 'Airfoil #200 in eigenspace')

plt.title("Airfoils")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(['Airfoil #200', 'Airfoil #200 in eigenspace'])

#### Exercise 
Measure reconstruction loss of out-of-sample airfoil `X_test` with different number of principal components in terms of 2-norm, conclude.

In [ ]:
# apply PCA
norms = []

for n in range(X_train.shape[1]):
    pca = PCA(n_components=(n+1))
    X_train_pca = pca.fit_transform(X_train)
    eigenvectors = pca.components_
    
    sample_mean = X_train.mean(axis=0)
    X_test_pcs = np.dot(np.dot(X_test - sample_mean, eigenvectors.T), eigenvectors) + sample_mean
    
    norm = # your code here
    norms.append(norm)

In [ ]:
fig = plt.figure(figsize=(6,6.5))
plt.title("Reconstruction loss")
plt.grid(linestyle="dotted")
plt.plot(norms)
plt.xlabel("# PCs")
plt.ylabel("2-norm")
plt.show()

### 2.2. MNIST

In [ ]:
mnist = fetch_openml("mnist_784", data_home="./data/mnist", version=1, as_frame=False)
train_size = 60000

X_train, X_test, y_train, y_test = \
mnist.data[:train_size,:], mnist.data[train_size:,:], \
mnist.target[:train_size].astype(int), mnist.target[train_size:].astype(int)

print("Dataset summary:\nSamples: {}, features: {}, classes: {}"
      .format(X_train.shape[0] + X_test.shape[0], X_train.shape[1], np.unique(y_train).shape[0]))

In [ ]:
# apply PCA
pca = PCA()
pca.fit(X_train)

#### Exercise

Plot expained and cumulative explained variance, choose sample dimensionality, state the rule used

In [ ]:
EV = # your code here
CEV = # your code here

In [ ]:
# plot EV/CEVs
fig = plt.figure(figsize=(12,5.25))

plt.subplot(121)
plt.title("Explained variance")
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.plot(EV)

plt.subplot(122)
plt.title("Cumulative explained variance")
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.axhline(linewidth=1, y=0.99, color='r')
plt.axhline(linewidth=1, y=0.95, color='r')
plt.axhline(linewidth=1, y=0.9, color='r')
plt.axhline(linewidth=1, y=0.8, color='r')
plt.plot(CEV)

Choose one of the digits $0 \dots 9$ and encode it in terms of different number of principal compoments.

In [ ]:
# select digit
X_digit = X_train[y_train==0]

# apply PCA
pca = PCA(n_components=4)
pca.fit(X_digit)
eigenvectors = pca.components_

In [ ]:
# encode vector in eigenbasis
sample_mean = X_digit.mean(axis=0)
X_digit_pcs = np.dot(np.dot(X_digit - sample_mean, eigenvectors.T), eigenvectors) + sample_mean

In [ ]:
# plot numbers
shape = (28, 28)
_, ax = plt.subplots(10, 10, figsize=(10, 10),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, a in enumerate(ax.flat):
    a.imshow(X_digit_pcs[i].reshape(shape), cmap=plt.cm.gray, interpolation="nearest")

#### Exercise 
Measure reconstruction loss of chosen digit within test data `X_test` with different number of principal components in terms of mean 2-norm and PSNR metric.

PSNR gives the ratio between the maximum possible power of a signal and the power of corrupting noise in decibels between image and its noisy approximation:

$$\textrm{PSNR}(x, y) := 20\log_{10} \left( \frac{MAX_I}{\sqrt{MSE}} \right) = 20\log_{10} \left( \frac{MAX_I}{\sqrt{\frac{1}{n} \sum (x - y)^2}} \right)$$

where $MAX_I$ is the maximum possible pixel value of the image. In our case it is 255.

In [ ]:
def psnr(x, y):
    # your code here

In [ ]:
assert isclose(psnr(np.ones(5), np.zeros(5)), 20 * np.log10(255), rel_tol=1e-3), "Check implementation of PSNR"

In [ ]:
norms = []
psnrs = []

range(2)

for n in list(map(lambda x: 2 ** x, range(10))):
    print("N", n)
    pca = PCA(n_components=(n+1))
    X_train_pca = pca.fit_transform(X_train)
    eigenvectors = pca.components_
    
    sample_mean = X_train.mean(axis=0)
    X_test_pcs = np.dot(np.dot(X_test - sample_mean, eigenvectors.T), eigenvectors) + sample_mean
    
    norm = # your code here
    psnr_metric = # your code here
    
    norms.append(norm)
    psnrs.append(psnr_metric)

In [ ]:
fig = plt.figure(figsize=(6,6.5))
plt.title("Reconstruction loss")
plt.grid(linestyle="dotted")
plt.plot(norms)
plt.xlabel("# PCs")
plt.ylabel("2-norm")
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6.5))
plt.title("Reconstruction loss")
plt.grid(linestyle="dotted")
plt.plot(psnrs)
plt.xlabel("# PCs")
plt.ylabel("PSNR")
plt.show()

## 3. PCs interpretation

### 3.1. US arrests

In [ ]:
# load data
X_arrests = np.loadtxt("./data/usarrests.csv", delimiter=",", skiprows=1, usecols=(1,2,3,4))
targets = list(np.genfromtxt('./data/usarrests.csv', delimiter=',', dtype='str', skip_header=True)[:,0])
y_arrests = [ x.replace('"', '') for x in targets ]
features_arrest = list(np.genfromtxt('./data/usarrests.csv', delimiter=',', dtype='str', skip_header=False)[0,1:])
features_arrest = [ x.replace('"', '') for x in features_arrest ]
#X_arrests, y_arrests

#### Exercise

Standartize data to zero mean and unit variance with `StandardScaler`

In [ ]:
# standartize data to zero mean and unit variance
X_arrests_std = # your code here

In [ ]:
pca = PCA()
pca.fit(X_arrests_std)

In [ ]:
# set eigenvectors to project data to
pc1, pc2 = 0, 1
eigenvector_x = pca.components_[pc1]
eigenvector_y = pca.components_[pc2]

# project data into PC space
xs = pca.transform(X_arrests_std)[:,pc1]
ys = pca.transform(X_arrests_std)[:,pc2]

# visualize projections
plt.figure(figsize=(10,10))

# arrows project features (ie columns from csv) as vectors onto PC axes
for i in range(eigenvector_x.shape[0]):
    plt.arrow(0, 0, eigenvector_x[i] * max(xs), eigenvector_y[i] * max(ys), color='r', width=0.0005)
    plt.text(eigenvector_x[i] * max(xs) + 0.05, eigenvector_y[i] * max(ys), features_arrest[i], color='r')

# circles project documents (ie rows from csv) as points onto PC axes
for i in range(xs.shape[0]):
    plt.plot(xs[i], ys[i], 'bo', alpha=0.25)
    plt.text(xs[i] - 0.05, ys[i] + 0.1, y_arrests[i], color='b', alpha=0.25)

plt.title("Biplot")
plt.xlabel("PC " + str(pc1 + 1))
plt.ylabel("PC " + str(pc2 + 1))
plt.xlim((-3.5, 3.5))
plt.ylim((-3.5, 3.5))
plt.grid(linestyle="dotted")
plt.show()

### 3.2. Wine dataset

In [ ]:
wine = load_wine()
X_wine = wine.data
y_wine = wine.target
features_wine = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
# X_wine, y_wine, features_wine

In [ ]:
X_wine_std = StandardScaler().fit_transform(X_wine)

pca = PCA()
pca.fit(X_wine_std)

In [ ]:
# set eigenvectors to project data to
pc1, pc2 = 0, 1
eigenvector_x = pca.components_[pc1]
eigenvector_y = pca.components_[pc2]

# project data into PC space
xs = pca.transform(X_wine_std)[:,pc1]
ys = pca.transform(X_wine_std)[:,pc2]

# visualize projections
plt.figure(figsize=(10,10))

# arrows project features (ie columns from csv) as vectors onto PC axes
for i in range(eigenvector_x.shape[0]):
    plt.arrow(0, 0, eigenvector_x[i] * max(xs), eigenvector_y[i] * max(ys), color='r', width=0.0005, alpha=0.5)
    plt.text(eigenvector_x[i] * max(xs) + 0.05, eigenvector_y[i] * max(ys), features_wine[i], color='r')

colors = ['r', 'g', 'b']
# circles project documents (ie rows from csv) as points onto PC axes
for i in range(xs.shape[0]):
    plt.plot(xs[i], ys[i], 'o', color=colors[y_wine[i]], alpha=0.25)

plt.title("Biplot")
plt.xlabel("PC " + str(pc1 + 1))
plt.ylabel("PC " + str(pc2 + 1))
plt.xlim((-4.5, 4.5))
plt.ylim((-4.5, 4.5))
plt.grid(linestyle="dotted")
plt.show()

## 4. PCA for decision making

- high-dimensional space is more dangerous to overfit complex models, than simple ones
- PC are orthogonal, decorellated features
    - may improve linear classifiers, if data multicolinearity problem
    - for complex decision boundaries classifiers, they can be a problem in even lower dimensions
- variable scaling, ensure your data is scaled to isotropic Gaussian $\sim \mathcal{N}(0, 1)$ or at least have similar scale (`MinMaxScaler`)
- you can estimate first $q$ PCs only, using iterative optimization, instead full eigenvalue/SVD decoposition and then taking top PC corresponing to largest eigenvalues

### 4.1. Eigenfaces

Eigenfaces as principal compoments and logistic regression classification

In [ ]:
# load Olivetti faces dataset
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

dataset = fetch_olivetti_faces(shuffle=True, random_state=1)
train_size = 350
X_train, X_test, y_train, y_test = dataset.data[:train_size,:], dataset.data[train_size:,:], dataset.target[:train_size], dataset.target[train_size:]
shape = (64, 64)
print("Dataset summary:\nSamples: {}, features: {}, classes: {}".format(dataset.data.shape[0], dataset.data.shape[1], np.unique(dataset.target).shape[0]))


In [ ]:
# plot faces
_, ax = plt.subplots(10, 10, figsize=(10, 10),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, a in enumerate(ax.flat):
    a.imshow(X_train[i].reshape(shape), cmap=plt.cm.gray, interpolation="nearest")

#### Exercise

Plot first 64 eigenfaces, comment the difference between the first and the last ones

In [ ]:
pca = PCA()
X_train_pca = pca.fit_transform(X_train)

In [ ]:
# plot first 64 components
_, ax = plt.subplots(8, 8, figsize=(10, 10),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, a in enumerate(ax.flat):
    a.imshow(pca.components_[i].reshape(shape), cmap=plt.cm.gray, interpolation="nearest")

#### Exercise
Encode any face in test dataset in eigenbasis of different powers, `n_components`= [2, 4, 8 16, 32, 64, $\dots$]

In [ ]:
# apply PCA
face_id = 1
pca = PCA(n_components=64)
X_train_pca = pca.fit_transform(X_train)
eigenvectors = pca.components_

sample_mean = X_train.mean(axis=0)
X_test_pcs = # your code here

In [ ]:
# plot
_, ax = plt.subplots(1, 2, figsize=(5, 5),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

ax[0].imshow(X_test[face_id].reshape(shape), cmap=plt.cm.gray, interpolation="nearest")
ax[1].imshow(X_test_pcs.reshape(shape), cmap=plt.cm.gray, interpolation="nearest")

#### Exercise 
Classify faces in original dimension with kNN, logistic regression and random forest, conclude.

In [ ]:
from sklearn.metrics import accuracy_score 

clf = # your code here
clf.fit(X_train, y_train)

acc_full_train = accuracy_score(clf.predict(X_train), y_train)
acc_full_test = accuracy_score(clf.predict(X_test), y_test)

acc_full_train, acc_full_test

#### Exercise

Perform classification in the space of reduced dimension of your choice, conclude.

In [ ]:
n_components = # your code here

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

clf = # your code here
clf.fit(X_train_pca, y_train)
accuracy_score(clf.predict(X_train_pca), y_train), accuracy_score(clf.predict(X_test_pca), y_test)

#### Exercise

- Perform classification for a whole range of dimensions, with a classifier of your choice.
- State the maximum number of principal components for this space, conclude. 
- Plot the classification performance.

In [ ]:
acc_train = []
acc_test = []

for n in # your code here:
    # your code here

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title("Classifier performance")
plt.xlabel("# PCs")
plt.ylabel("Accuracy")
plt.grid(linestyle="dotted")
plt.axhline(linewidth=1, y=acc_full_test, color='r')
plt.plot(acc_train, 'b')
plt.plot(acc_test, 'g')
plt.legend(['Best on test in original dimension', 'Train in reduced dimension', 'Test in reducted dimension'])
plt.show()